<a href="https://colab.research.google.com/github/SSenitha/CCS3052_Advance_DSA/blob/Sandaru-s-Branch/adjacencyMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Load data from the repository
!wget https://raw.githubusercontent.com/SSenitha/CCS3052_Advance_DSA/refs/heads/main/Cities_of_SriLanka.csv

--2025-09-11 08:03:02--  https://raw.githubusercontent.com/SSenitha/CCS3052_Advance_DSA/refs/heads/main/Cities_of_SriLanka.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 264501 (258K) [text/plain]
Saving to: ‘Cities_of_SriLanka.csv.1’

Cities_of_SriLanka. 100%[===================>] 258.30K  1.46MB/s    in 0.2s    

2025-09-11 08:03:02 (1.46 MB/s) - ‘Cities_of_SriLanka.csv.1’ saved [264501/264501]



In [13]:
#Import necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import stats
from sklearn.neighbors import NearestNeighbors


In [32]:
# Read the dataset and output the total count
df = pd.read_csv('/content/Cities_of_SriLanka.csv')
print(f"Total rows count: {df['city id'].count()}")
df.head()

Total rows count: 2155


,city id,district_id,name_en,name_si,name_ta,sub_name_en,sub_name_si,sub_name_ta,postcode,latitude,longitude
0,1,1,Akkaraipattu,අක්කරපත්තුව,அக்கரைப்பற்று,NULL,NULL,NULL,32400.0,7.218428,81.854116
1,2,1,Ambagahawatta,අඹගහවත්ත,அம்பகஹவத்த,NULL,NULL,NULL,90326.0,7.301756,81.674729
2,3,1,Ampara,අම්පාර,அம்பாறை,NULL,NULL,NULL,32000.0,7.301756,81.674729
3,4,1,Bakmitiyawa,බක්මිටියාව,பக்மிடியாவ,NULL,NULL,NULL,32024.0,7.029632,81.680205
4,5,1,Deegawapiya,දීඝවාපිය,தீகவாபி,NULL,NULL,NULL,32006.0,7.301756,81.674729


In [33]:
df = df.drop_duplicates(subset=['latitude', 'longitude'], keep='first').reset_index(drop=True)
print(f"Total rows count after dropping duplicates: {df['city id'].count()}")

Total rows count after dropping duplicates: 1919


In [34]:
df = df.drop(columns=["district_id","name_si","name_ta","sub_name_en","sub_name_si","sub_name_ta","postcode"])

In [35]:
df.head()

,city id,name_en,latitude,longitude
0,1,Akkaraipattu,7.218428,81.854116
1,2,Ambagahawatta,7.301756,81.674729
2,4,Bakmitiyawa,7.029632,81.680205
3,7,Digamadulla Weeragoda,7.390125,81.696588
4,8,Dorakumbura,7.358870,81.301428


#KNN Approach

In [16]:
import math

def dist(lat1, lon1, lat2, lon2):
    R = 6371   # Radius of Earth

    # Convert degrees into radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    #Calculate the linear distance between two points
    distance = dlat ** 2 + dlon ** 2
    distance = math.sqrt(distance)

    # Calculate spherical distance
    a = math.asin(distance/(2*R))
    c = R * 2 * math.asin(math.sqrt(a))

    return c


In [21]:
num_cities = len(df)
# Create a matrix filled with zeros
adj_matrix_dense = np.zeros((num_cities, num_cities))

# Calculate distances using the dist function
for i in range(num_cities):
    for j in range(num_cities):
        if i != j:
            lat1, lon1 = df['latitude'][i], df['longitude'][i]
            lat2, lon2 = df['latitude'][j], df['longitude'][j]
            adj_matrix_dense[i, j] = dist(lat1, lon1, lat2, lon2)

print("Shape of the dense adjacency matrix:", adj_matrix_dense.shape)
# Display a small portion of the dense matrix
print("\nSmall portion of the dense adjacency matrix (first 10x10):")
print(adj_matrix_dense[:10, :10])

Shape of the dense adjacency matrix: (1919, 1919)

Small portion of the dense adjacency matrix (first 10x10):
[[ 0.          6.63232845  7.55545411  7.19859907 11.26136494  8.98978275
   8.84675427  6.29982929  7.90783408  5.94831077]
 [ 6.63232845  0.          7.78010196  4.49940599  9.16430046  6.20394095
   9.11781122  6.40301922  7.10140592  6.37903887]
 [ 7.55545411  7.78010196  0.          8.95839665 10.56456152  9.64949837
   4.81690525  9.39380995 10.31088151  9.22925734]
 [ 7.19859907  4.49940599  8.95839665  0.          9.38902986  5.64768591
  10.15329695  5.5175263   5.64471669  5.68857151]
 [11.26136494  9.16430046 10.56456152  9.38902986  0.          7.7091858
  11.12771611 10.88946018 10.67449512 10.96247176]
 [ 8.98978275  6.20394095  9.64949837  5.64768591  7.7091858   0.
  10.65977214  7.84337721  7.38341552  7.99707463]
 [ 8.84675427  9.11781122  4.81690525 10.15329695 11.12771611 10.65977214
   0.         10.54715166 11.38054372 10.39528164]
 [ 6.29982929  6.4030192